In [1]:
import torch
from torch_geometric.data import HeteroData

# Initialize the node features for each type of node
# For this example, let's say each node A has 5 features, and nodes B1 and B2 do not have features
num_nodes_A = 1000
num_nodes_B1 = 500
num_nodes_B2 = 500

x_A = torch.randn((num_nodes_A, 5))  # 5 features for each node A
x_B1 = torch.zeros((num_nodes_B1, 0))  # no features for nodes B1
x_B2 = torch.zeros((num_nodes_B2, 0))  # no features for nodes B2

# Initialize the edges for each type of edge
# For this example, let's say we have 2000 edges between nodes A and B1, 
# and 3000 edges between nodes B1 and B2
num_edges_A_B1 = 2000
num_edges_B1_B2 = 3000

edge_index_A_B1 = torch.randint(0, num_nodes_A, (2, num_edges_A_B1))
edge_index_B1_B2 = torch.randint(0, num_nodes_B1, (2, num_edges_B1_B2))

# Initialize the labels for each type of edge
# For this example, let's say we're doing binary classification, so labels are 0 or 1
y_A_B1 = torch.randint(0, 2, (num_edges_A_B1,))
y_B1_B2 = torch.randint(0, 2, (num_edges_B1_B2,))

# Create a HeteroData object
data = HeteroData()

# Add node data
data['A'].x = x_A
data['B1'].x = x_B1
data['B2'].x = x_B2

# Add edge data
data['A', 'connects', 'B1'].edge_index = edge_index_A_B1
data['A', 'connects', 'B1'].y = y_A_B1

data['B1', 'connects', 'B2'].edge_index = edge_index_B1_B2
data['B1', 'connects', 'B2'].y = y_B1_B2

In [8]:
x_B1

tensor([], size=(500, 0))

In [7]:
data['B1', 'connects', 'B2'].edge_index

tensor([[173, 436, 416,  ...,  48,  74, 306],
        [289, 254, 185,  ...,  68, 143, 399]])

In [9]:
data['A', 'connects', 'B1'].y

tensor([0, 1, 1,  ..., 0, 1, 1])

In [2]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, to_hetero , SAGEConv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore") 

path_work = "/media/concha-eloko/Linux/PPT_clean"
graph_data = torch.load(f'{path_work}/graph_file.1107.pt')

In [9]:
graph_data.edge_index_dict

{('B1',
  'infects',
  'A'): tensor([[   0,    0,    1,  ..., 7728, 7729, 7730],
         [   0,    0,    0,  ..., 4528, 4529, 4377]]),
 ('B2',
  'expressed',
  'B1'): tensor([[ 1909,  2314,   122,  ...,  3605,  3606,  3607],
         [    0,     0,     1,  ..., 11336, 11337, 11338]]),
 ('A',
  'harbors',
  'B1'): tensor([[   0,    0,    0,  ..., 4528, 4529, 4377],
         [   0,    0,    1,  ..., 7728, 7729, 7730]])}

In [19]:
transform = T.RandomLinkSplit(
    num_val=0.1, 
    num_test=0.2, 
    #disjoint_train_ratio=...,  
    neg_sampling_ratio=1.0,  
    add_negative_train_samples=True, 
    edge_types=("B1", "infects", "A"),
    rev_edge_types=("A", "harbors", "B1"), 
)

train_data, val_data, test_data = transform(graph_data)

train_loader = LinkNeighborLoader(
    data=train_data,  
    num_neighbors= [-1],  
    edge_label_index=(("B1", "infects", "A"), train_data["B1", "infects", "A"].edge_label_index),
    edge_label=train_data["B1", "infects", "A"].edge_label,
    batch_size=128,
    shuffle=True,
)

val_loader = LinkNeighborLoader(
    data=val_data,  
    num_neighbors= [-1],  
    edge_label_index=(("B1", "infects", "A"), val_data["B1", "infects", "A"].edge_label_index),
    edge_label=val_data["B1", "infects", "A"].edge_label,
    batch_size=128,
    shuffle=True,
)

test_loader = LinkNeighborLoader(
    data=test_data,  
    num_neighbors= [-1],  
    edge_label_index=(("B1", "infects", "A"), test_data["B1", "infects", "A"].edge_label_index),
    edge_label=test_data["B1", "infects", "A"].edge_label,
    batch_size=128,
    shuffle=True,
)

In [18]:
graph_data.x_dict["B2"][2]

tensor([ 0.0188,  0.0681, -0.0091,  ..., -0.1246,  0.0700,  0.0653])

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(-1, hidden_channels, add_self_loops=False)
        self.conv2 = GCNConv(-1, out_channels, add_self_loops=False)
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def forward(self, x, edge_index):
        edge_feat_B1 = x["B1"][edge_index[("B1", "infects", "A")][0]]
        edge_feat_A = x["A"][edge_index[("B1", "infects", "A")][1]]
        return (edge_feat_B1 * edge_feat_A).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.gnn_B2_B1 = GNN(hidden_channels, out_channels)
        self.gnn_B1_A = GNN(hidden_channels, out_channels)
        self.classifier = Classifier()
    def forward(self, graph_data):
        # Propagate B2 features to B1
        x_B2 = graph_data['B2'].x
        edge_index_B2_B1 = graph_data[('B2', 'expressed', 'B1')].edge_index
        x_B1_from_B2 = self.gnn_B2_B1(x_B2, edge_index_B2_B1)
        # Propagate new B1 features to A
        edge_index_B1_A = graph_data[('B1', 'infects', 'A')].edge_index
        x_A_from_B1 = self.gnn_B1_A(x_B1_from_B2, edge_index_B1_A)
        # Classification based on new features
        x = {'B1': x_B1_from_B2, 'A': x_A_from_B1}
        pred = self.classifier(x, edge_index_B1_A)
        
        return pred

In [ ]:
# *****************************************************************************
# The model : Concat
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(-1, hidden_channels, add_self_loops=False)
        self.conv2 = GCNConv(-1, out_channels, add_self_loops=False)
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.linear = torch.nn.Linear(in_channels * 2, 1)
    def forward(self, x, edge_index):
        edge_feat_B1 = x["B1"]
        edge_feat_A = x["A"]
        edge_features = torch.cat([edge_feat_B1[edge_index[0]], edge_feat_A[edge_index[1]]], dim=-1)
        return self.linear(edge_features)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.gnn_B2_B1 = GNN(hidden_channels, out_channels)
        self.gnn_B1_A = GNN(hidden_channels, out_channels)
        self.classifier = Classifier(out_channels)

    def forward(self, graph_data):
        # Propagate B2 features to B1
        x_B2 = graph_data['B2'].x
        edge_index_B2_B1 = graph_data[('B2', 'expressed', 'B1')].edge_index
        x_B1_from_B2 = self.gnn_B2_B1(x_B2, edge_index_B2_B1)
        # Propagate new B1 features to A
        x_B1 = x_B1_from_B2
        edge_index_B1_A = graph_data[('B1', 'infects', 'A')].edge_index
        x_A_from_B1 = self.gnn_B1_A(x_B1, edge_index_B1_A)
        # Classification based on new features
        x = {'B1': x_B1_from_B2, 'A': x_A_from_B1}
        pred = self.classifier(x, edge_index_B1_A)
        return pred